In [ ]:
import torch
import numpy as np
import open3d as o3d
import sys
sys.path.append('../src/')
import os
sys.path.append('/home/ruslan/datasets/data/python/data/')
from asl_laser import Dataset

# only needed for tutorial, monkey patches visualization
sys.path.append('/home/ruslan/subt/thirdparty/Open3D/examples/python/')
import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ

### Load data

In [ ]:
from depth_correction.depth_cloud import DepthCloud

sequences_names = [
    'apartment',
    'eth',
    'gazebo_winter',
    'gazebo_summer',
    'plain',
    'stairs',
    'wood_summer',
    'wood_autumn'
]

data_path = '/home/ruslan/datasets/data/asl_laser/'

data = Dataset(path=os.path.join(data_path, sequences_names[0]))

points = torch.as_tensor(data.local_cloud(id=0)[::50, :])
dc = DepthCloud.from_points(pts=points)

# we need incidence angles for depth correction
dc.estimate_incidence_angles()

### Create model

In [ ]:
from depth_correction.model import Linear, Polynomial

model = Polynomial()
# model = Linear()

# The optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

device = torch.device('cuda:0')
model = model.to(device)
dc = dc.to(device)

### Optimization loop

In [ ]:
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

# Number of optimization steps
Niter = 800
plot_period = 200

for i in range(Niter):
    # Initialize optimizer
    optimizer.zero_grad()
    
    dc_deform = model(dc)
    
    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    src_pts = dc_deform.to_points()[None]
    tgt_pts = dc.to_points()[None]
    loss, _ = chamfer_distance(src_pts, tgt_pts)
    
    # Plot results
    if i % plot_period == 0:
        print('Loss:', loss.item())
        dc_deform.visualize()
        
    # Optimization step
    loss.backward()
    optimizer.step()